<a href="https://colab.research.google.com/github/LisburnLad/ArtificialLife/blob/master/Energy2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai-v3/'

Mounted at /content/gdrive


In [3]:
!curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [0]:
from fastai.vision import *

In [5]:
path_energy = Config.data_path()/'energy'
path_energy.mkdir(parents=True, exist_ok=True) 

!mkdir -p ~/.kaggle/
!cp '/content/gdrive/My Drive/fastai-v3/kaggle.json' ~/.kaggle/    

!pip install kaggle --upgrade
!kaggle competitions download -c ashrae-energy-prediction -p {path_energy} 

Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.6)
100% 120M/120M [00:02<00:00, 47.3MB/s] 

 92% 81.0M/88.4M [00:01<00:00, 43.7MB/s]
100% 88.4M/88.4M [00:01<00:00, 53.9MB/s]
 97% 162M/167M [00:03<00:00, 48.0MB/s]
100% 167M/167M [00:03<00:00, 50.4MB/s]
  0% 0.00/1.27M [00:00<?, ?B/s]
100% 1.27M/1.27M [00:00<00:00, 83.9MB/s]
  0% 0.00/2.53M [00:00<?, ?B/s]
100% 2.53M/2.53M [00:00<00:00, 83.4MB/s]
  0% 0.00/44.5k [00:00<?, ?B/s]
100% 44.5k/44.5k [00:00<00:00, 44.9MB/s]


In [0]:
google_drive = Path(base_dir)

In [0]:
# !rm 'energy_utils.py'
# !cp '/content/gdrive/My Drive/fastai-v3/energy_utils.py' .
# from energy_utils import *

In [0]:
import zipfile
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()


# unzip the specified file and delete the zip file on completion
def unzip_and_delete( aPath, aSourceZip ):
  with zipfile.ZipFile(str(aPath/aSourceZip), 'r') as zip_ref:
      zip_ref.extractall(str(aPath))  
  os.remove(str(aPath/aSourceZip))      
  

#Based on this great kernel https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65
def reduce_mem_usage(df):
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in df.columns:
        if df[col].dtype != object:  # Exclude strings            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",df[col].dtype)            
            # make variables for Int, max and min
            IsInt = False
            mx = df[col].max()
            mn = df[col].min()
            print("min for this col: ",mn)
            print("max for this col: ",mx)
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(df[col]).all(): 
                NAlist.append(col)
                df[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = df[col].fillna(0).astype(np.int64)
            result = (df[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        df[col] = df[col].astype(np.uint8)
                    elif mx < 65535:
                        df[col] = df[col].astype(np.uint16)
                    elif mx < 4294967295:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)    
            # Make float datatypes 32 bit
            else:
                df[col] = df[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",df[col].dtype)
            print("******************************")
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df, NAlist  


def create_dataframe( path_energy, aMainZip, aWeatherZip ):

  unzip_and_delete( path_energy, aMainZip )
  unzip_and_delete( path_energy, aWeatherZip )

  building_df = pd.read_csv(path_energy/"building_metadata.csv")
  weather_train = pd.read_csv(path_energy/"weather_train.csv")
  main_df = pd.read_csv(path_energy/"train.csv")   

  main_df = main_df.merge(building_df, left_on = "building_id", right_on = "building_id", how = "left")
  main_df = main_df.merge(weather_train, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"]) 
  
  del building_df
  del weather_train

  main_df["timestamp"] = pd.to_datetime(main_df["timestamp"])
  main_df["hour"] = main_df["timestamp"].dt.hour
  main_df["day"] = main_df["timestamp"].dt.day
  main_df["weekday"] = main_df["timestamp"].dt.weekday
  main_df["month"] = main_df["timestamp"].dt.month
  main_df = main_df.drop("timestamp", axis = 1)

  main_df["primary_use"] = label_encoder.fit_transform(main_df["primary_use"])

  main_df, NAlist = reduce_mem_usage(main_df)

  return main_df    

In [0]:
train = create_dataframe( path_energy, "train.csv.zip", "weather_train.csv.zip" )

In [12]:
train.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,weekday,month
0,0,0,0.0,0,0,7432,2008,0,25.0,6,20.0,-2,1019.700012,0,0.0,0,1,4,1
1,1,0,0.0,0,0,2720,2004,0,25.0,6,20.0,-2,1019.700012,0,0.0,0,1,4,1
2,2,0,0.0,0,0,5376,1991,0,25.0,6,20.0,-2,1019.700012,0,0.0,0,1,4,1
3,3,0,0.0,0,0,23685,2002,0,25.0,6,20.0,-2,1019.700012,0,0.0,0,1,4,1
4,4,0,0.0,0,0,116607,1975,0,25.0,6,20.0,-2,1019.700012,0,0.0,0,1,4,1


In [0]:
from fastai.tabular import *

In [0]:
cat_names = ["building_id", "site_id", "primary_use", "floor_count", "hour", "day", "weekday", "month", "meter"]
cont_names = ["square_feet", "year_built", "air_temperature", "cloud_coverage", "dew_temperature"]
drop_cols = ["precip_depth_1_hr", "sea_level_pressure", "wind_direction", "wind_speed"]
dep_var = 'meter_reading'

procs = [FillMissing, Categorify, Normalize]

In [0]:
data = (TabularList.from_df(train, path=path_energy, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                          #  .add_test(test)
                           .databunch())

In [16]:
data.show_batch(rows=10)

building_id,site_id,primary_use,floor_count,hour,day,weekday,month,meter,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,target
818,8,9,1,22,5,1,4,0,-0.8888,-0.7092,1.0597,-0.8748,0.1685,0.5833
1092,13,6,0,10,17,5,12,2,-0.0223,-0.7092,-2.5826,1.1418,-2.3135,3773.44
651,4,8,5,19,7,5,5,0,1.6930,2.0650,-0.0127,1.1418,0.3373,40.8
985,9,6,0,3,7,0,3,0,-0.6233,-0.7092,0.3386,-0.8908,0.7245,17.0
701,5,0,4,6,4,4,3,0,0.1784,1.0119,-1.2699,-0.8908,-0.7647,50.4
251,2,0,0,10,13,2,1,0,-0.8109,-0.7092,-0.8354,-0.8908,-0.7052,30.13
1383,15,0,0,3,17,0,10,0,-0.0185,1.0376,0.0890,1.1418,0.7840,88.6
993,9,0,0,4,16,2,11,2,2.7386,-0.7092,0.0336,-0.8908,0.4465,1250.5
1073,13,6,0,12,4,3,2,0,0.7368,-0.7092,-2.4347,-0.8270,-2.0355,129.0
1296,14,6,0,14,29,4,1,1,-0.8445,-0.7092,-1.5565,1.1418,-1.2611,117.053


In [0]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [18]:
learn.fit(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time


RuntimeError: ignored

In [0]:
learn.save(google_drive/'energy-2')
learn.export(google_drive/'energy-2.pkl')

In [20]:
import gc
del train

gc.collect()

108

In [21]:
test_df = create_dataframe( path_energy, "test.csv.zip", "weather_test.csv.zip" ):

SyntaxError: ignored

In [0]:
test_image_list = TabularList.from_df(test_df, path=path_energy, cat_names=cat_names, cont_names=cont_names, procs=procs)


In [0]:
learn = load_learner(google_drive/'energy-2.pkl')

In [0]:
learn.data.add_test(test_image_list)
preds,_ = learn.get_preds(DatasetType.Test)

In [0]:
res = np.concatenate(preds)

In [0]:
pd.DataFrame(res).describe()

In [0]:
res.shape

In [0]:
unzip_and_delete( path_energy, "sample_submission.csv.zip" )

In [0]:
sub = pd.read_csv(path_energy/"sample_submission.csv")

In [0]:
sub["meter_reading"] = res

In [0]:
sub.to_csv(path_energy/"submission.csv", index = False)

In [0]:
!kaggle competitions submit ashrae-energy-prediction -f {path_energy/'submission.csv'} -m "energy 2"